In [8]:
import numpy as np
import pandas as pd
import requests as rq
from citipy import citipy
from config import weather_api_key
from datetime import datetime

In [2]:
lats = np.random.uniform(low=-90.000, high=90.000, size = 2000)
longs = np.random.uniform(low=-180.000, high=180.000, size = 2000)
latlongs = zip(lats, longs)
coordinates = list(latlongs)

In [3]:
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    if city not in cities:
        cities.append(city)

In [4]:
cityData = []

recordCount = 1
setCount = 1

In [5]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&appid=" + weather_api_key
for i, city in enumerate(cities):
    if (i % 50 == 0 and i >= 50):
        setCount += 1
        recordCount = 1
    cityurl = url + "&q=" + city.replace(" ","+")
    recordCount += 1
    try:
        # Parse the JSON and retrieve data.
        cityWeather = rq.get(cityurl).json()

        # Parse out the needed data.
        cityLat = cityWeather["coord"]["lat"]
        cityLong = cityWeather["coord"]["lon"]
        cityMaxTemp = cityWeather["main"]["temp_max"]
        cityHumidity = cityWeather["main"]["humidity"]
        cityClouds = cityWeather["clouds"]["all"]
        cityWind = cityWeather["wind"]["speed"]
        cityCountry = cityWeather["sys"]["country"]
        cityDesc = cityWeather["weather"][0]["description"]
        cityData.append({"City": city.title(),
                          "Lat": cityLat,
                          "Lng": cityLong,
                          "Max Temp": cityMaxTemp,
                          "Humidity": cityHumidity,
                          "Cloudiness": cityClouds,
                          "Wind Speed": cityWind,
                          "Country": cityCountry,
                          "Current Description": cityDesc})

    # If an error is experienced, skip the city.
    except:
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

-----------------------------
Data Retrieval Complete      
-----------------------------


In [9]:
cityDatadf = pd.DataFrame(cityData)
cityDatadf = cityDatadf[["City","Country","Lat","Lng", "Max Temp", 
                         "Humidity", "Cloudiness", "Wind Speed", "Current Description"]]
cityDatadf.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Haines Junction,CA,60.7522,-137.5108,31.95,55,100,0.00,overcast clouds
1,Descalvado,BR,-21.9039,-47.6194,65.28,91,97,8.14,overcast clouds
2,Namatanai,PG,-3.6667,152.4333,86.45,62,49,6.55,scattered clouds
3,Dikson,RU,73.5069,80.5464,27.84,94,100,25.88,light snow
4,Lebu,CL,-37.6167,-73.6500,52.12,87,0,16.64,clear sky


In [10]:
outputDataFile = "WeatherPy_Database.csv"
cityDatadf.to_csv(outputDataFile, index_label="City_ID")